In [1]:
import glob

raw_files = glob.glob('GermInnC/**/*.txt')

In [3]:
import os
import codecs
import magic

data = []
error = 0

for file in raw_files:
    m_file = magic.Magic(mime_encoding=True)
    file_encoding = m_file.from_buffer(file)
    file = file.encode(file_encoding).decode('UTF-8')
    genre, period, region, year, title = os.path.basename(file).split('_')[:5]
    with open(file, 'rb') as f:
        raw_data = f.read()
    m = magic.Magic(mime_encoding=True)
    encoding = m.from_buffer(raw_data)
    if encoding != 'unknown-8bit':
        text = raw_data.decode(encoding=encoding).encode('UTF-8').decode('UTF-8')
    else:
        error += 1
        continue
    data.append({
        'text': text,
        'title': title,
        'genre': genre,
        'period': period,
        'region': region,
        'year': year
    })

print(f'Failed to read {error} files')

Failed to read 42 files


In [4]:
import pandas as pd

df = pd.DataFrame.from_dict(data)
df.to_csv('dataset.csv', encoding='UTF-8')

In [5]:
data[0]

{'text': 'Aus Stadt und Land.\n\n(Trauergottesdienst.) Zur Erinnerung an den Todestag Gr. Majestät des Kaisers Ferdinand I., sowie aus Anlaß der ersten Jahreswende seit dem Ableben Gr. k. u. k· Hoheit des durchlauchtigsten Herrn Erzherzogs Franz Ferdinand von Oesterreich-Este und Höchstdessen Gemahlin wird am Montag, dem 28. Juni, um 9 Uhr vormittags ein Trauergottesdienst in der St. Jakob Pfarrkirche abgehalten werden.\n(Zum Thronfolgerbesuch.) Vom Besuche des Erzherzogs Thronfolger in unserer Stadt sind bei Formator Stainer in der Maria Theresienstraße einige vorzüglich gelungene Photographiekarten zu haben, welche sich als hübsche Erinnerung an den schönen Tag von selbst empfehlen. — Dem Innsbrucker Friseur Matthias Müller ist anläßlich der Anwesenheit des Erzherzog\'s-Thronfolgers eine besondere Ehre zuteil geworden. Müller\' erhielt den Auftrag, den Thronfolger rasieren und frisieren zu dürfen, da es dem Thronfolger aber infolge seines kurz bemessen und vollständig aufgeteilten Au

In [81]:
tagged_files = glob.glob('GermInnC Tagged/**/*.txt')

In [125]:
import os

tagged_data = []
for file in tagged_files:
    genre, period, region, year, title = os.path.basename(file).split('_')[:5]
    tokens = []
    pos_tags = []
    lemmas = []
    with open(file, 'r', encoding='UTF-8') as f:
        for line in f:
            line = line.rstrip()
            try:
                token, pos_tag, lemma = line.split('\t')
            except ValueError:
                continue
            tokens.append(token)
            pos_tags.append(pos_tag)
            lemmas.append(lemma)
        
    tagged_data.append({
        'tokens': " ".join(tokens),
        'pos_tags': " ".join(pos_tags),
        'lemmas': " ".join(lemmas),
        'title': title,
        'genre': genre,
        'period': period,
        'region': region,
        'year': year
    })

In [126]:
tagged_data[0]

{'tokens': 'Lokales und Vermischtes . Der Kaiser in Breslau . Der Kaiser , der bekanntlich heute den Feierlichkeiten anläßlich der Vermählung der Gräfin Armgard zu Stolberg = Wernigerode mit dem Grafen Platen zu Hallermund in Landeshut beiwohnen wird , begibt sich gleich nach der Trauung nach Breslau , wo er heute Abend eintrifft . Der Kaiser wird sich , wie uns ein Privat = Telegramm aus Breslau meldet , sofort nach dem fürstbischhöflichen Palais begeben , um beim Kardinal Kopp das Souper einzunehmen . Die Rückreise des Kaisers nach Berlin erfolgt am Dienstag Nachmittag . – Die Kaiserin hat ihre Teilnahme an der Hochzeitsfeierlichkeit wegen leichter Unpäßlichkeit abgesagt . Gestern Vormittag wohnte der Kaiser dem Gottesdienst zu Potsdam bei und empfing um 12 ¾ Uhr den Staatsminister Budde . – Gestern Abend begab sich der Kaiser nach dem königlichen Schauspielhaus , um der Vorstellung von Blumenthals „ Mann wir altern “ und Rofstands „ Die Romantischen “ beizuwohnen . Im Schauspielhaus

In [127]:
import pandas as pd

df = pd.DataFrame.from_dict(tagged_data)
df.to_csv('tagged_dataset.csv', encoding='UTF-8')